Process the data...

In [ ]:
data_dir = ".\\data\\"

#filename = "2018-06-20-10-44-11.tdms"
#keyence_filename= "2018-06-20-10-44-18-KEYENCE.tdms"

# SLC-2445, air, slow
#filename = "2018-07-12-18-36-41.tdms"
#keyence_filename= "2018-07-12-18-36-56-KEYENCE.tdms"

# SLC-2445, air, fast
#filename = "2018-07-12-18-51-08.tdms"
#keyence_filename= "2018-07-12-18-51-17-KEYENCE.tdms"

#filename = "2018-07-17-10-17-29.tdms"  # SLC-2445, vac, fast
#filename = "2018-07-17-09-45-52.tdms" # SLC-2445, vac, slow
#keyence_filename = None

# SLC-1730, air, slow
#filename = "2018-07-25-10-58-32.tdms"
#keyence_filename= "2018-07-25-10-58-24-KEYENCE.tdms"

# SLC-1730, air, fast
#filename = "2018-07-25-11-13-44.tdms"
#keyence_filename= "2018-07-25-11-13-50-KEYENCE.tdms"

# SLC-1730, air, fast
#filename = "2018-07-26-10-30-38.tdms"
#keyence_filename= "2018-07-26-10-31-06-KEYENCE.tdms"

# SLC-1730, air, slow
#filename = "2018-07-26-10-44-20.tdms"
#keyence_filename= "2018-07-26-10-44-40-KEYENCE.tdms"

# SLC-1730, vac, 24hrs
#filename = "2018-07-26-14-39-24.tdms"
#keyence_filename= None

# SLC-2445, vac, 72hrs
filename = "2018-07-27-14-45-18.tdms"
keyence_filename= None

#duty test SLC-1730
filename = "2018-09-27-09-10-21.tdms"
keyence_filename = None

In [ ]:
import batch_utils
import data_tools

In [ ]:
from ipywidgets import FloatProgress
from IPython.display import display

import numpy as np
from pprint import pprint
import datetime
from time import time

from importlib import reload
reload(batch_utils)
reload(data_tools)

dry_run = False

full_data, info = batch_utils.load_data(filename, keyence_filename, suffix="full_data")

batch_utils.print_lengths(full_data, "Full dataset:")

print("Round the position to 3dp")
full_data['position']['round'] = list(np.around(full_data['position']['data'], 2))

print("Initialise the transition time")
full_data['set_point']['trans'] = [None,] * len(full_data['set_point']['time'])

print("Calculate data rates")
start = time()
rate = data_tools.data_rates(full_data)
duration = time() - start
print("\tTook {}s".format(duration))
pprint(rate)


print("Chopping up the data!")
slices = list(range(0, len(full_data['set_point']['time']), 1000))
#slices = list(range(0, 5000, 1000))
print("Got {} slices".format(len(slices)))
slices.append(-2)
#slices.append(5000)
#pprint(slices)

progress = FloatProgress(max=len(slices)-1)
display(progress)

js = [min(j+100, len(full_data['set_point']['time'])) for j in slices[1:]]

for i, j in zip(slices, js):
    
    print("\n\nGetting the next slice {} to {}".format(i, j+1))
    batch_utils.print_lengths(full_data, "Full dataset:")
    
    limits = [full_data['set_point']['time'][i], full_data['set_point']['time'][j]]
    print("Using limits:")
    pprint(limits)
    
    data = dict(zip(full_data.keys(), [data_tools.truncate(limits, full_data[k], find_mode={'seek':False}) for k in full_data]))
    batch_utils.print_lengths(data, "Truncated dataset:")
    
    print("Done!")
    

    print("Calculating transition time")
    p = FloatProgress()
    display(p)

    if not dry_run: 
        start = time()
        batch_utils.calculate_trans(data, p)
        duration = time() - start
        print("\tTook {}s".format(duration))

    print("Done!")


    print("Calculating differences")
    p = FloatProgress()
    display(p)

    diff_pairs = [
        (['position', 'data'],            ['set_point', 'data'])
        ]

    if keyence_filename is not None:
        diff_pairs.extend([
        (['keyence', ['data1', 'data2']], ['set_point', 'data']),
        (['keyence', ['data1', 'data2']], ['position',  'data']),
        ])

    if not dry_run:
        start = time()
        data_tools.calculate_differences(diff_pairs, data, progress=p)
        duration = time() - start
        print("\tTook {}s".format(duration))

    print("Done!")


    print("Calculating masks")
    p = FloatProgress()
    display(p)

    mask_pairs = [('position', 'set_point')]
    jitter = rate['position']

    if keyence_filename is not None:
        mask_pairs.extend([('keyence',  'set_point')])
        jitter = max([rate['position'], rate['keyence']])

    mask_margin = [datetime.timedelta(seconds = - (jitter)),
                   datetime.timedelta(seconds = -(0.5 - jitter))]

    if not dry_run: 
        start = time()
        data_tools.generate_mask(mask_pairs, mask_margin, data, progress=p)
        duration = time() - start
        print("\tTook {}s".format(duration))

    print("Done!")  
    
    
    print("Aggregating data")
    
    k_agg = ['position', 'data_set_point_data']
    k_win = ['set_point', 'time']

    p = FloatProgress()
    display(p)
    
    if not dry_run:
        start = time()
        data_tools.aggregate_data(k_agg, k_win, data, p)
        duration = time() - start
        print("\tTook {}s".format(duration))

    print("Done!")
    
    print("Storing the data")
    batch_utils.store_data(data, filename, keyence_filename, suffix="proc_data_part_{}-{}".format(i,j), info=info)
    print("Done!")
    
    progress.value += 1

print("Finished processing!")
progress.value = progress.max



import copy

print("Merging data")
p = FloatProgress()
display(p)

merged_data = None

for i, j in zip(slices, slices[1:]):
    print("\n\nGetting the next slice {} to {}".format(i, j))
    
    start = time()
    
    data, info = batch_utils.load_data(filename, keyence_filename, suffix="proc_data_part_{}-{}".format(i,j))
    
    if merged_data is None:
        merged_data = copy.deepcopy(data)
    
    batch_utils.merge_data(merged_data, data, p)
    batch_utils.print_lengths(merged_data, "After merge:")
    
    duration = time() - start
    print("\tTook {}s".format(duration))

print("Done!")

print("Storing the processed data")
if not dry_run: 
    batch_utils.store_data(merged_data, filename, keyence_filename, suffix="proc_data", info=info)
print("Done!")

print("The process is complete!!")

In [ ]:
batch_utils.store_data(merged_data, filename, keyence_filename, suffix="proc_data", info=info)

In [ ]:
jitter